# Mapreader Workshops 2024

----

First check you have the correct version of MapReader: v1.3.2

This can be downloaded from pypi using `pip install mapreader==1.3.2` or by checking out the repo at [this commit](https://github.com/Living-with-machines/MapReader/releases/tag/v1.3.2)

In [1]:
import mapreader
assert mapreader.__version__ == '1.3.2'

-------------

# Classify

We can also use our ``ClassifierContainer`` to infer labels on new datasets with a pre-trained/fine-tuned model.

We are going to look at how to use our railspace model (https://huggingface.co/Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace) to predict labels on patches.To do this, we need to load the model from huggingface and then pass it in as the ``model`` argument in our ``ClassifierContainer``.

We need to use the ``timm`` library to set up our model. You can see this is shown in the top right corner of the [model page](https://huggingface.co/Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace) where it says ``</> Use in timm``.

Other models will have different instructions, e.g. [resnet-50](https://huggingface.co/microsoft/resnet-50) should be loaded with the ``transformers`` library.

In [2]:
try:
    import timm
except ImportError: # timm isn't installed by default, so you might need to install it
    !pip install timm
    import timm

In [3]:
my_model = timm.create_model("hf_hub:Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_building", pretrained=True)

After setting up the model, we can then set up our ``ClassifierContainer``. This is done in the same way as before, except this time we pass ``my_model`` as the model argument.

In [4]:
from mapreader import ClassifierContainer

The below will make sure that the model training/inference runs as as fast as possible on your machine by using CUDA (GPU) or MPS if they are available.

This ``device`` variable can then be fed into the ``ClassifierContainer``.

In [5]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [6]:
my_classifier = ClassifierContainer(
    model=my_model,
    labels_map={0:"no", 1:"building"}, # manually set this, you can find it in the model card - https://huggingface.co/Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace
    dataloaders=None,
)

[INFO] Device is set to cpu
[INFO] Initializing model.


## Infer

The model can now be used to infer, or predict, the labels of "unseen" patches.

To show how inference works, we will predict the labels on patches from just one parent image. 

We will do this by creating a ``subset_patch_df`` from our previously saved ``patch_df.csv``.
Our new ``subset_patch_df`` will only contain the information of patches from ``map_75650661.png``.

In [7]:
import pandas as pd

patch_df = pd.read_csv("./patch_df.csv", index_col=0)  # load our patch_df.csv file

subset_patch_df = patch_df[
    patch_df["parent_id"] == "map_75650661.png"
]  # filter for our chosen parent image
subset_patch_df.head()

,parent_id,image_path,shape,pixel_bounds,coordinates,crs,polygon,mean_pixel_R,mean_pixel_G,mean_pixel_B,mean_pixel_A,std_pixel_R,std_pixel_G,std_pixel_B,std_pixel_A
image_id,,,,,,,,,,,,,,,
patch-0-0-149-149-#map_75650661.png#.png,map_75650661.png,/Users/rwood/LwM/mapreader/MapReader/worked_ex...,"(149, 149, 4)","(0, 0, 149, 149)","(-4.2572021484375, 55.86670862718037, -4.25560...",EPSG:4326,POLYGON ((-4.255603551864624 55.86670862718037...,0.759666,0.740528,0.712193,1.0,0.181756,0.181556,0.177755,0.0
patch-0-149-149-298-#map_75650661.png#.png,map_75650661.png,/Users/rwood/LwM/mapreader/MapReader/worked_ex...,"(149, 149, 4)","(0, 149, 149, 298)","(-4.2572021484375, 55.865811287362945, -4.2556...",EPSG:4326,POLYGON ((-4.255603551864624 55.86581128736294...,0.775035,0.755652,0.726691,1.0,0.173966,0.173666,0.170123,0.0
patch-0-298-149-447-#map_75650661.png#.png,map_75650661.png,/Users/rwood/LwM/mapreader/MapReader/worked_ex...,"(149, 149, 4)","(0, 298, 149, 447)","(-4.2572021484375, 55.86491394754552, -4.25560...",EPSG:4326,POLYGON ((-4.255603551864624 55.86491394754552...,0.802534,0.781693,0.751058,1.0,0.176199,0.175807,0.171707,0.0
patch-0-447-149-596-#map_75650661.png#.png,map_75650661.png,/Users/rwood/LwM/mapreader/MapReader/worked_ex...,"(149, 149, 4)","(0, 447, 149, 596)","(-4.2572021484375, 55.86401660772808, -4.25560...",EPSG:4326,POLYGON ((-4.255603551864624 55.86401660772808...,0.804019,0.783748,0.752186,1.0,0.172955,0.172673,0.168803,0.0
patch-0-596-149-745-#map_75650661.png#.png,map_75650661.png,/Users/rwood/LwM/mapreader/MapReader/worked_ex...,"(149, 149, 4)","(0, 596, 149, 745)","(-4.2572021484375, 55.863119267910655, -4.2556...",EPSG:4326,POLYGON ((-4.255603551864624 55.86311926791065...,0.784714,0.763500,0.732548,1.0,0.190242,0.189836,0.185525,0.0


> __**NOTE**__: MapReader can be used to predict the labels on entire datasets and so creating a ``subset_patch_df`` is not needed in most use cases.

### Create a dataset (``infer``) from our ``subset_patch_df``

In [8]:
from mapreader import PatchDataset

In [9]:
infer = PatchDataset(patch_df, transform="val", patch_paths_col="image_path")

### Load dataset into ``my_classifier``

In the same way as we did in the last notebook, we need to load our dataset into our classifier container.

In [12]:
my_classifier.load_dataset(infer, "infer_building")

### Run model inference

__**YOUR TURN**__: Run inference on your ``"infer_railspace"`` dataset

See [here](https://mapreader.readthedocs.io/en/latest/User-guide/Classify/Infer.html#infer) in docs.

In [13]:
my_classifier.inference("infer_building")

[INFO] Each step will pass: ['infer_building'].
2024-06-03 12:15:34 599-JY5FK6 [INFO] infer_building -- 1/1 --   16/10470 (  0.2% ) -- 
2024-06-03 12:15:58 599-JY5FK6 [INFO] infer_building -- 1/1 --   96/10470 (  0.9% ) -- 
2024-06-03 12:16:22 599-JY5FK6 [INFO] infer_building -- 1/1 --  176/10470 (  1.7% ) -- 
2024-06-03 12:16:46 599-JY5FK6 [INFO] infer_building -- 1/1 --  256/10470 (  2.4% ) -- 
2024-06-03 12:17:09 599-JY5FK6 [INFO] infer_building -- 1/1 --  336/10470 (  3.2% ) -- 
2024-06-03 12:17:33 599-JY5FK6 [INFO] infer_building -- 1/1 --  416/10470 (  4.0% ) -- 
2024-06-03 12:17:55 599-JY5FK6 [INFO] infer_building -- 1/1 --  496/10470 (  4.7% ) -- 
2024-06-03 12:18:20 599-JY5FK6 [INFO] infer_building -- 1/1 --  576/10470 (  5.5% ) -- 
2024-06-03 12:18:49 599-JY5FK6 [INFO] infer_building -- 1/1 --  656/10470 (  6.3% ) -- 
2024-06-03 12:19:17 599-JY5FK6 [INFO] infer_building -- 1/1 --  736/10470 (  7.0% ) -- 
2024-06-03 12:19:41 599-JY5FK6 [INFO] infer_building -- 1/1 --  816/1047

Remember to save your results!

In [14]:
my_classifier.save_predictions("infer_building")

[INFO] Saved predictions to infer_building_predictions_patch_df.csv.


### Save results to metadata

To add the predictions back into a ``MapImages`` object, we simply need to load our predictions csv file as metadata.

Since we have started a new notebook, we can create a new ``MapImages`` object by loading our patches.

In [16]:
from mapreader import load_patches

In [17]:
my_maps = load_patches(
    "./patches_100_meters/*png", parent_paths="./maps/*png"
)

0it [00:00, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/10470 [00:00<?, ?it/s]

In [18]:
my_maps.add_metadata("./infer_building_predictions_patch_df.csv", tree_level="patch")

In [19]:
my_maps.add_shape()

[INFO] Add shape, tree level: parent


We can use the ``.show_parent()`` method to see how our predictions look on our parent map sheet (``map_74488700.png``).

In [ ]:
my_maps.show_parent(
    "map_75650661.png",
    column_to_plot="pred",
    vmin=0,
    vmax=1,
    alpha=0.5,
    patch_border=False,
)

And the ``.convert_images()`` method to save our results.

In [ ]:
parent_df, patch_df = my_maps.convert_images(save=True, save_format="excel") # here we are saving to xlsx so we don't change our "*.csv" files from before!

We can also save our outputs as a ``geojson`` file using the ``.save_patches_to_geojson()`` method. We'll call the file ``"railspace_patches.geojson"``.

> _**NOTE**_: This will require you to convert your patch coordinates into a polygon format. If these aren't already available, they can be added using the ``.add_patch_polygons()`` method.

In [20]:
my_maps.add_patch_polygons()
my_maps.save_patches_to_geojson("building_patches.geojson")

  0%|          | 0/10470 [00:00<?, ?it/s]

Beyond MapReader, these outputs can be used to generate interesting visualizations in other tools.

For example, here are two visualizations of the rail space data from [our paper]:

- https://felt.com/map/MapReader-Launch-Event-map-Urban-Areas-and-Rail-space-9AqftKrvPTlWfwOGkdkCGkD
- https://maps.nls.uk/projects/mapreader/index.html#zoom=6.0&lat=56.00000&lon=-4.00000

# Documentation

Please refer to the [MapReader documentation](https://mapreader.readthedocs.io/en/latest/) for more information.